In [14]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, log_loss
from scipy.stats import norm

from typing import Tuple, Dict, List

In [3]:
# global variables
PATH_TO_DATA_FILE : str = 'drive/MyDrive/data/data.csv' # путь к файлу с данными

In [4]:
data = pd.read_csv(PATH_TO_DATA_FILE)
data.head()

,date_time,zone_id,banner_id,oaid_hash,campaign_clicks,os_id,country_id,banner_id0,rate0,g0,coeff_sum0,banner_id1,rate1,g1,coeff_sum1,impressions,clicks
0,2021-09-27 00:01:30.000000,0,0,5664530014561852622,0,0,0,1240,0.067,0.035016,-7.268846,0,0.010,0.049516,-5.369901,1,1
1,2021-09-26 22:54:49.000000,1,1,5186611064559013950,0,0,1,1,0.002,0.054298,-2.657477,269,0.004,0.031942,-4.449220,1,1
2,2021-09-26 23:57:20.000000,2,2,2215519569292448030,3,0,0,2,0.014,0.014096,-3.824875,21,0.014,0.014906,-3.939309,1,1
3,2021-09-27 00:04:30.000000,3,3,6262169206735077204,0,1,1,3,0.012,0.015232,-3.461357,99,0.006,0.050671,-3.418403,1,1
4,2021-09-27 00:06:21.000000,4,4,4778985830203613115,0,1,0,4,0.019,0.051265,-4.009026,11464230,6.790,0.032005,-2.828797,1,1


# Предобработка
удаляю столбцы, не используемые в данном задании
- campaign_clicks -- т.к. не нужен по условию
- impressions -- т.к. в первой дз было выявлено, что этот столбец не несет информации
- oaid_hash, rate0, rate1 -- удаляю, т.к. они не используются ни при подаче в модель, ни при вычислении OPE (oaid_hash можно было бы оставить, но тогда нужно будет заново подбирать гиперпараметры модели)

In [5]:
data = data.drop(columns = ['campaign_clicks', 'impressions', 'oaid_hash', 'rate0', 'rate1'])
data.head()

,date_time,zone_id,banner_id,os_id,country_id,banner_id0,g0,coeff_sum0,banner_id1,g1,coeff_sum1,clicks
0,2021-09-27 00:01:30.000000,0,0,0,0,1240,0.035016,-7.268846,0,0.049516,-5.369901,1
1,2021-09-26 22:54:49.000000,1,1,0,1,1,0.054298,-2.657477,269,0.031942,-4.449220,1
2,2021-09-26 23:57:20.000000,2,2,0,0,2,0.014096,-3.824875,21,0.014906,-3.939309,1
3,2021-09-27 00:04:30.000000,3,3,1,1,3,0.015232,-3.461357,99,0.050671,-3.418403,1
4,2021-09-27 00:06:21.000000,4,4,1,0,4,0.051265,-4.009026,11464230,0.032005,-2.828797,1


Проведем небольшой анализ столбцов, которые будем использовать для вычислений

In [6]:
# проверяем, есть ли в таблице null, если да -- то в каких столбцах и в каком количестве
null_columns=data.columns[data.isnull().any()]
if len(null_columns) == 0:
  print("Dataframe does not consists null values")
else:
  print("Number of null rows in columns")
  print(data[null_columns].isnull().sum())

del null_columns

Number of null rows in columns
g0               69
coeff_sum0       69
g1            19744
coeff_sum1    19744
dtype: int64


Как будем предобрабатывать данные:
Для подачи в модель будут использоваться столбцы: zone_id, banner_id, os_id,	country_id, hour(вытащим из даты). И целевой столбец clicks.

Остальные столбцы будем использовать для вычисления OPE: banner_id0,	g0,	coeff_sum0,	banner_id1,	g1,	coeff_sum1.	

Для подачи в модель подготовим данные аналогично первому дз:
1.  zone_id, banner_id, os_id,	country_id --  категориальные признаки, выполним encoding для них
2. Из информации о дате и времени оставим столбец часы, используем encoding и для него

Доп:
3. У столбцов zone_id, banner_id достаточно большое количество значений, при использовании one-hot encoding разреженные таблицы не влезают в память. Поэтому закодирую редкие значения этих признаков одним классом(-1, т.к. подобное значение точно не встречается в столбцах). Порог отношения к одному классу выбран исходя из ограничений памяти

Также отфильтруем test данные, и только после выполним их подготовку для подачи в модель:
1. Удалим те строки, где banner_id != banner_id0
2. Удалим те строки, где g0, coeff_sum0, g1, coeff_sum1 равны null -- их мы не сможем использовать в вычислениях

Train данные фильтровать не будем


In [7]:
# эта часть должна была быть функцией, но из-за локальных переменных вычисления не влезали в память
columns_and_quantiles_to_group_rare_values = {'zone_id' : 0.4, 'banner_id' : 0.2}
columns_to_ope = ['banner_id0', 'g0', 'coeff_sum0', 'banner_id1', 'g1', 'coeff_sum1']

data['date_time'] = pd.to_datetime(data['date_time'], format = "%Y-%m-%d %H:%M:%S.%f")
data['date'] = data['date_time'].dt.date.astype('str')
data['hour'] = data['date_time'].dt.hour

data = data.drop(columns = ['date_time'])

for column_name, quant in columns_and_quantiles_to_group_rare_values.items():
  values = data[column_name].value_counts()
  qua = values.quantile(quant)
  indexes = values[values<qua].index
  data.loc[data[column_name].isin(indexes), column_name] = -1

# подготовим encoder для кодировки категориальных столбцов
categorical_features = ['zone_id', 'banner_id', 'os_id', 'country_id', 'hour']
encoder = preprocessing.OneHotEncoder(handle_unknown='ignore').fit(data[categorical_features])

train_data = data[data['date'] != '2021-10-02'].drop(columns=['date'])
test_data = data[data['date'] == '2021-10-02'].drop(columns=['date'])

# очищаю data чтоб уместиться в памяти
del data

# подготавливаем данные для подачи в модель
train_X = encoder.transform(train_data[categorical_features])
train_X = train_X.asformat('csr') 
train_y = train_data.clicks
del train_data 

# отфильтруем test данные
test_size_before_filter = test_data.shape
test_data = test_data[test_data['banner_id'] == test_data['banner_id0']]
test_data = test_data.dropna()
print(f"filter test data. shape before = {test_size_before_filter}; shape after = {test_data.shape}")

# таблица с данными, которые будем использовать для вычислений
test_ope_data = test_data[columns_to_ope]
# подготавливаем данные, где banner_id = banner_id0
test_X0 = encoder.transform(test_data[categorical_features])
test_X0 = test_X0.asformat('csr')
# оставим y для проверки качества модели с помощью метрик, использованных в дз 1
test_y0 = test_data.clicks
# подготавливаем данные, где banner_id = banner_id1
test_data = test_data.drop(columns = 'banner_id')
test_data.rename(columns={'banner_id1': 'banner_id'}, inplace=True)
test_X1 = encoder.transform(test_data[categorical_features])
test_X1 = test_X1.asformat('csr')

del test_data


filter test data. shape before = (2128978, 12); shape after = (1885557, 12)


# Подготовка модели
Данные подготовлены, обучим модель на train с теми гиперпараметрами, которые подобрали в первой дз.

In [8]:
def create_model(regularization: str = 'l2', regul_const : float = 1.0):
    return LogisticRegression(penalty = regularization, solver = 'liblinear', C = regul_const, random_state=42)

In [9]:
def calculate_metrics(y_true, y_pred):
  print(f"Roc= {roc_auc_score(y_true, y_pred[:,1])}")
  print(f"Log loss = {log_loss(y_true, y_pred)}")

In [10]:
# best model according hw1
model = create_model('l2', 0.001)
model.fit(train_X, train_y)

LogisticRegression(C=0.001, random_state=42, solver='liblinear')

In [12]:
print("results for our model:")
test_y0_pred = model.predict_proba(test_X0)
calculate_metrics(test_y0, test_y0_pred)

results for our model:
Roc= 0.7786851269731156
Log loss = 0.13660911128897985


In [13]:
del train_X
del train_y

# Вычисление clipped ips

Составим формулу для вычисления $\pi_0$, $\pi_1$:

Рассмотрим случайные величины 
$\xi_0 \sim N(c_0, g_0^2);~~~~~~~\xi_1 \sim N(c_1, g_1^2)$

$$\pi(\xi_0|c_0, g_0, c_1, g_1) = Pr(\xi_0 \ge \xi_1) \\ 
\pi(\xi_1|c_0, g_0, c_1, g_1) = 1- Pr(\xi_0 \ge \xi_1)$$

$$Pr(\xi_0\ge\xi_1) = Pr(\xi_0-\xi_1\ge 0) = Pr(\xi_1-\xi_0 \le 0) = F_{\xi_1-\xi_0}(0)$$

$$\xi_1-\xi_0 \sim N(c_1-c_0, g_1^2+g_0^2)$$

$$\rightarrow Pr(\xi_0\ge\xi_1) = F_{\xi_1-\xi_0}(0)$$


In [17]:
def calculate_pr(c0, g0, c1, g1):
  diff_loc = c1 - c0
  diff_scale = np.sqrt(g0**2+g1**2)
  return norm.cdf(0, diff_loc, diff_scale)

In [22]:
def logit(p, eps = 1e-15):
  return np.log(p+eps)-np.log(1-p+eps)

In [24]:
# вычислим \pi_0
pi_0 = calculate_pr(test_ope_data['coeff_sum0'], test_ope_data['g0'], test_ope_data['coeff_sum1'], test_ope_data['g1'])
# вычислим новые coeff_sum для \pi_1
coeff_sum0_new = logit(test_y0_pred[:,1])
test_y1_pred = model.predict_proba(test_X1)
coeff_sum1_new = logit(test_y1_pred[:, 1])
# вычислим \pi_1
pi_1 =  calculate_pr(coeff_sum0_new, test_ope_data['g0'], coeff_sum1_new, test_ope_data['g1'])

In [25]:
def clipped_ips(r, pi_0, pi_1, lambd = 10, eps = 1e-15):
  return np.mean(r*np.minimum(pi_1/(pi_0+eps), lambd))

Вычислим clipped IPS:

In [26]:
reward = test_y0
clipped_ips(reward, pi_0, pi_1)

0.0754135201045623